In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("juhibhojani/house-price")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/house-price


In [ ]:
df=pd.read_csv("/kaggle/input/house-price/house_prices.csv")

In [ ]:
df.head()

,Index,Title,Description,Amount(in rupees),Price (in rupees),location,Carpet Area,Status,Floor,Transaction,...,facing,overlooking,Society,Bathroom,Balcony,Car Parking,Ownership,Super Area,Dimensions,Plot Area
0,0,1 BHK Ready to Occupy Flat for sale in Srushti...,"Bhiwandi, Thane has an attractive 1 BHK Flat f...",42 Lac,6000.0,thane,500 sqft,Ready to Move,10 out of 11,Resale,...,NaN,NaN,Srushti Siddhi Mangal Murti Complex,1,2,NaN,NaN,NaN,NaN,NaN
1,1,2 BHK Ready to Occupy Flat for sale in Dosti V...,One can find this stunning 2 BHK flat for sale...,98 Lac,13799.0,thane,473 sqft,Ready to Move,3 out of 22,Resale,...,East,Garden/Park,Dosti Vihar,2,NaN,1 Open,Freehold,NaN,NaN,NaN
2,2,2 BHK Ready to Occupy Flat for sale in Sunrise...,Up for immediate sale is a 2 BHK apartment in ...,1.40 Cr,17500.0,thane,779 sqft,Ready to Move,10 out of 29,Resale,...,East,Garden/Park,Sunrise by Kalpataru,2,NaN,1 Covered,Freehold,NaN,NaN,NaN
3,3,1 BHK Ready to Occupy Flat for sale Kasheli,This beautiful 1 BHK Flat is available for sal...,25 Lac,NaN,thane,530 sqft,Ready to Move,1 out of 3,Resale,...,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN
4,4,2 BHK Ready to Occupy Flat for sale in TenX Ha...,"This lovely 2 BHK Flat in Pokhran Road, Thane ...",1.60 Cr,18824.0,thane,635 sqft,Ready to Move,20 out of 42,Resale,...,West,"Garden/Park, Main Road",TenX Habitat Raymond Realty,2,NaN,1 Covered,Co-operative Society,NaN,NaN,NaN


In [ ]:
df.shape

(187531, 21)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187531 entries, 0 to 187530
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Index              187531 non-null  int64  
 1   Title              187531 non-null  object 
 2   Description        184508 non-null  object 
 3   Amount(in rupees)  187531 non-null  object 
 4   Price (in rupees)  169866 non-null  float64
 5   location           187531 non-null  object 
 6   Carpet Area        106858 non-null  object 
 7   Status             186916 non-null  object 
 8   Floor              180454 non-null  object 
 9   Transaction        187448 non-null  object 
 10  Furnishing         184634 non-null  object 
 11  facing             117298 non-null  object 
 12  overlooking        106095 non-null  object 
 13  Society            77853 non-null   object 
 14  Bathroom           186703 non-null  object 
 15  Balcony            138596 non-null  object 
 16  Ca

In [ ]:
df.describe()

,Index,Price (in rupees),Dimensions,Plot Area
count,187531.000000,1.698660e+05,0.0,0.0
mean,93765.000000,7.583772e+03,NaN,NaN
std,54135.681003,2.724171e+04,NaN,NaN
min,0.000000,0.000000e+00,NaN,NaN
25%,46882.500000,4.297000e+03,NaN,NaN
50%,93765.000000,6.034000e+03,NaN,NaN
75%,140647.500000,9.450000e+03,NaN,NaN
max,187530.000000,6.700000e+06,NaN,NaN


In [ ]:
df = df.drop('Index', axis = 'columns')

In [ ]:
df.rename(columns = { 'Amount(in rupees)': 'Amount_in_rupees',
                     'Price (in rupees)': 'Price_in_repees',
                     'Carpet Area': 'Carpet_area_in_sqft'}, inplace = True)

In [ ]:
null_percent = df.isnull().mean() * 100
null_percent

,0
Title,0.000000
Description,1.612000
Amount_in_rupees,0.000000
Price_in_repees,9.419776
location,0.000000
Carpet_area_in_sqft,43.018488
Status,0.327946
Floor,3.773776
Transaction,0.044259
Furnishing,1.544811


In [ ]:
# Find columns where null percentage is greater than 50
miss_value_50_perc = null_percent[null_percent > 50]

# Drop the columns from the DataFrame
df = df.drop(columns=miss_value_50_perc.index)

In [ ]:
df.shape

(187531, 15)

In [ ]:
## Handling Carpet Area:
def convert_to_sqft(area):
    try:
        if pd.notnull(area):
            if 'sqft' in area:
                area= float(area.replace(' sqft',''))
            else:
                area=float(area.replace(' sqm',''))*10.7639
            return area
    except ValueError:
        return np.nan

df['Carpet_area_in_sqft']=df['Carpet_area_in_sqft'].apply(convert_to_sqft)

In [ ]:
## Handling the amount in rupees

def convert_rupees(amount_str):
    try:
        parts=amount_str.split()
        amount=float(parts[0])

        if len(parts)>1:
            unit=parts[1].strip()
            if unit=='Lac':
                amount*=100000
            elif unit=='Cr':
                amount*=10000000
        return amount
    except(ValueError,IndexError):
        return None

df['Amount_in_rupees']= df['Amount_in_rupees'].apply(convert_rupees)

In [ ]:
df.describe()

,Amount_in_rupees,Price_in_repees,Carpet_area_in_sqft
count,1.778470e+05,1.698660e+05,101322.000000
mean,1.198134e+07,7.583772e+03,1267.967442
std,3.943827e+07,2.724171e+04,3131.148649
min,1.000000e+05,0.000000e+00,1.000000
25%,4.840000e+06,4.297000e+03,804.000000
50%,7.800000e+06,6.034000e+03,1068.000000
75%,1.450000e+07,9.450000e+03,1500.000000
max,1.400300e+10,6.700000e+06,709222.000000


In [ ]:
categorical_features = df.select_dtypes(include = ['object']).columns
categorical_features.tolist()

['Title',
 'Description',
 'location',
 'Status',
 'Floor',
 'Transaction',
 'Furnishing',
 'facing',
 'overlooking',
 'Bathroom',
 'Balcony',
 'Ownership']

In [ ]:
for categorical_feature in categorical_features :
    if df[categorical_feature].isnull().sum() != 0 :
        df[categorical_feature].replace(np.nan, df[categorical_feature].mode()[0], inplace=True)


<ipython-input-18-e603aa106c74>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[categorical_feature].replace(np.nan, df[categorical_feature].mode()[0], inplace=True)


In [ ]:
df.isnull().sum()

,0
Title,0
Description,0
Amount_in_rupees,9684
Price_in_repees,17665
location,0
Carpet_area_in_sqft,86209
Status,0
Floor,0
Transaction,0
Furnishing,0


In [ ]:
numaric_features = df.select_dtypes(include = ['float64']).columns
numaric_features.tolist()

['Amount_in_rupees', 'Price_in_repees', 'Carpet_area_in_sqft']

In [ ]:
for numaric_feature in numaric_features :
    if df[numaric_feature].isnull().sum() != 0 :
        df[numaric_feature].replace(np.nan, df[numaric_feature].mean(), inplace=True)

<ipython-input-21-683f7382af61>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[numaric_feature].replace(np.nan, df[numaric_feature].mean(), inplace=True)


In [ ]:
df.isnull().sum()

,0
Title,0
Description,0
Amount_in_rupees,0
Price_in_repees,0
location,0
Carpet_area_in_sqft,0
Status,0
Floor,0
Transaction,0
Furnishing,0


In [ ]:
# Calculate correlation matrix
correlation_matrix = df.select_dtypes(include=['int64', 'float64']).corr()

# Display correlation matrix
print(correlation_matrix)

                     Amount_in_rupees  Price_in_repees  Carpet_area_in_sqft
Amount_in_rupees             1.000000         0.760839             0.045551
Price_in_repees              0.760839         1.000000             0.008464
Carpet_area_in_sqft          0.045551         0.008464             1.000000


In [ ]:
# Status has only one value for all example that's why remove it.
df.drop(['Status', 'Title', 'Description'], axis = 'columns', inplace = True)

In [ ]:
df.head()


,Amount_in_rupees,Price_in_repees,location,Carpet_area_in_sqft,Floor,Transaction,Furnishing,facing,overlooking,Bathroom,Balcony,Ownership
0,4200000.0,6000.000000,thane,500.0,10 out of 11,Resale,Unfurnished,East,Main Road,1,2,Freehold
1,9800000.0,13799.000000,thane,473.0,3 out of 22,Resale,Semi-Furnished,East,Garden/Park,2,2,Freehold
2,14000000.0,17500.000000,thane,779.0,10 out of 29,Resale,Unfurnished,East,Garden/Park,2,2,Freehold
3,2500000.0,7583.771885,thane,530.0,1 out of 3,Resale,Unfurnished,East,Main Road,1,1,Freehold
4,16000000.0,18824.000000,thane,635.0,20 out of 42,Resale,Unfurnished,West,"Garden/Park, Main Road",2,2,Co-operative Society


In [ ]:
from sklearn.preprocessing import LabelEncoder

def label_encode_multiple(df, columns):
    encoder = LabelEncoder()
    for column in columns:
        df[column] = encoder.fit_transform(df[column])
    return df

label_encode_columns = ['Transaction','location', 'Furnishing', 'facing', 'overlooking', 'Ownership']
df = label_encode_multiple(df, label_encode_columns)

In [ ]:
df['Floor'].value_counts()

,count
Floor,
2 out of 4,19510
1 out of 4,11985
3 out of 4,8792
1 out of 3,7093
4 out of 4,6253
...,...
32 out of 33,1
Lower Basement out of 23,1
Ground out of 29,1


In [ ]:
# Split 'Floor' into two separate columns: current_floor and total_floors
df[['current_floor', 'total_floors']] = df['Floor'].str.split(' out of ', expand=True)

# Display the DataFrame with the new columns
print(df[['Floor', 'current_floor', 'total_floors']])

               Floor current_floor total_floors
0       10 out of 11            10           11
1        3 out of 22             3           22
2       10 out of 29            10           29
3         1 out of 3             1            3
4       20 out of 42            20           42
...              ...           ...          ...
187526    2 out of 4             2            4
187527    4 out of 6             4            6
187528    1 out of 3             1            3
187529    2 out of 2             2            2
187530   5 out of 13             5           13

[187531 rows x 3 columns]


In [ ]:
encoding_map = {
    'Ground': 0,
    'Upper Basement': -1,
    'Lower Basement': -2,

}

# Apply the encoding to the 'Ownership' column
df['current_floor'] = df['current_floor'].replace(encoding_map)

In [ ]:
df['current_floor'] = df['current_floor'].astype(int)


In [ ]:
df['total_floors'] = df['total_floors'].fillna(df['total_floors'].mode()[0])

# Convert 'total_floors' column to integer data type
df['total_floors'] = df['total_floors'].astype(int)

In [ ]:
df.drop('Floor', axis = 'columns', inplace = True)

In [ ]:
df['Bathroom'] = df['Bathroom'].replace({"> 10" : 11})
df['Bathroom'].value_counts()

,count
Bathroom,
2,93835
3,55781
1,18654
4,15600
5,3343
6,209
7,35
11,35
10,14


In [ ]:
df['Bathroom'] = df['Bathroom'].astype(int)

In [ ]:
df['Balcony'] = df['Balcony'].replace({"> 10" : 11})
df['Balcony'].value_counts()
df['Balcony'] = df['Balcony'].astype(int)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187531 entries, 0 to 187530
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Amount_in_rupees     187531 non-null  float64
 1   Price_in_repees      187531 non-null  float64
 2   location             187531 non-null  int64  
 3   Carpet_area_in_sqft  187531 non-null  float64
 4   Transaction          187531 non-null  int64  
 5   Furnishing           187531 non-null  int64  
 6   facing               187531 non-null  int64  
 7   overlooking          187531 non-null  int64  
 8   Bathroom             187531 non-null  int64  
 9   Balcony              187531 non-null  int64  
 10  Ownership            187531 non-null  int64  
 11  current_floor        187531 non-null  int64  
 12  total_floors         187531 non-null  int64  
dtypes: float64(3), int64(10)
memory usage: 18.6 MB


In [ ]:
df.head()

,Amount_in_rupees,Price_in_repees,location,Carpet_area_in_sqft,Transaction,Furnishing,facing,overlooking,Bathroom,Balcony,Ownership,current_floor,total_floors
0,4200000.0,6000.000000,67,500.0,3,2,0,7,1,2,1,10,11
1,9800000.0,13799.000000,67,473.0,3,1,0,0,2,2,1,3,22
2,14000000.0,17500.000000,67,779.0,3,2,0,0,2,2,1,10,29
3,2500000.0,7583.771885,67,530.0,3,2,0,7,1,1,1,1,3
4,16000000.0,18824.000000,67,635.0,3,2,7,1,2,2,0,20,42


In [ ]:
from sklearn.preprocessing import StandardScaler

numeric_df = df[['Price_in_repees', 'Carpet_area_in_sqft']]

scaler = StandardScaler()
numeric_df_standardized = scaler.fit_transform(numeric_df)

df_standardized = pd.DataFrame(numeric_df_standardized, columns=numeric_df.columns)

df_dropped_numeric = df.drop(columns=['Price_in_repees', 'Carpet_area_in_sqft'])
df_combined = pd.concat([df_dropped_numeric, df_standardized], axis=1)

df_combined.head()

,Amount_in_rupees,location,Transaction,Furnishing,facing,overlooking,Bathroom,Balcony,Ownership,current_floor,total_floors,Price_in_repees,Carpet_area_in_sqft
0,4200000.0,67,3,2,0,7,1,2,1,10,11,-6.108617e-02,-0.333677
1,9800000.0,67,3,1,0,0,2,2,1,3,22,2.397217e-01,-0.345408
2,14000000.0,67,3,2,0,0,2,2,1,10,29,3.824695e-01,-0.212453
3,2500000.0,67,3,2,0,7,1,1,1,1,3,1.052378e-16,-0.320642
4,16000000.0,67,3,2,7,1,2,2,0,20,42,4.335362e-01,-0.275020


In [ ]:
X = df_combined.drop(columns=['Amount_in_rupees'])
y = df_combined['Amount_in_rupees']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
models = {
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'XGBRegressor': XGBRegressor(),
}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    scoree = model.score(X_test,y_test)

    print(f'{model_name}:')
    print(f'R-squared: {r2:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
    print(f'Accuracy of Model:{scoree:.2f}')
    print('****************************************')

Random Forest:
R-squared: 0.71
Mean Absolute Error (MAE): 770127.61
Root Mean Squared Error (RMSE): 14785994.05
Accuracy of Model:0.71
****************************************
Gradient Boosting:
R-squared: 0.78
Mean Absolute Error (MAE): 1993252.35
Root Mean Squared Error (RMSE): 12691303.73
Accuracy of Model:0.78
****************************************
XGBRegressor:
R-squared: 0.63
Mean Absolute Error (MAE): 1037242.33
Root Mean Squared Error (RMSE): 16538916.53
Accuracy of Model:0.63
****************************************
